In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np

df = pd.read_csv("mnist_train.csv")
df_test = pd.read_csv("mnist_test.csv")

y_df_train = df["label"]
X_df_train = df.drop('label', axis=1)
y_df_test = df_test["label"]
X_df_test = df_test.drop('label', axis=1)

y_train = torch.tensor(y_df_train.values)
X_train = torch.tensor(X_df_train.values).float()
y_test = torch.tensor(y_df_test.values)
X_test = torch.tensor(X_df_test.values).float()

In [34]:
'''class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
learning_rate = 1e-3
batch_size = 64
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)'''
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        self.conv1 = Conv2d(in_channels=1, out_channels=20, kernel_size=(5, 5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=20, out_channels=50,kernel_size=(5, 5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=800, out_features=500)
        self.relu3 = ReLU()
        
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=500, out_features=10)
        self.logSoftmax = LogSoftmax(dim=1)

        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        
        x = self.fc2(x)
        logits = self.logSoftmax(x)
        
        return logits

model = NeuralNetwork()
learning_rate = 1e-3
batch_size = 64
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [40]:
def train_loop(X_data, y_data, model, loss_fn, optimizer):
    size = len(X_data)
    X_data = X_data.reshape(60000, 1, 28, 28)
    X_split = torch.split(X_data, batch_size, dim=0)
    y_split = torch.split(y_data, batch_size, dim=0)
    for X, y in zip(X_split, y_split):
       # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_size % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test_loop(X, y, model, loss_fn):
    size = len(X)
    num_batches = size / batch_size
    test_loss, correct = 0, 0
    with torch.no_grad():
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    loss_fn = nn.CrossEntropyLoss()    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [41]:
epochs = 100
for t in range(epochs):
    print("Epoch {t+1}\n-------------------------------")
    train_loop(X_train, y_train, model, loss_fn, optimizer)
    test_loop(X_test, y_test, model, loss_fn)
print("Done!")

Epoch {t+1}
-------------------------------


RuntimeError: Given groups=1, weight of size [6, 1, 5, 5], expected input[1, 64, 28, 28] to have 1 channels, but got 64 channels instead

In [20]:
from PIL import Image

test_num = 0
pred = model(X_test)

while(test_num < 60000):
    test_num += 1
    if(pred[test_num].argmax(0) != y_df_test[test_num]):
        if(input("Would you like to see the next failed Image? (y/n): ") == "y"):
            X_test_img = X_df_test.values[test_num ].reshape(28, 28)
            X_test_img = np.array(X_test_img).astype(np.uint8) # Create a PIL image

            img = Image.fromarray( X_test_img )
            img.show() 
    
            print("Test Number " + str(test_num) + " failed: ")
            print(pred[test_num])
            print(y_test[test_num])
        else:
            break



Would you like to see the next failed Image? (y/n): y
Test Number 149 failed: 
tensor([-3.1103,  4.1549,  6.8161,  8.6029,  3.4779,  2.4971,  0.1209, -2.7147,
         1.3510,  7.7360], grad_fn=<SelectBackward0>)
tensor(2)
Would you like to see the next failed Image? (y/n): y
Test Number 247 failed: 
tensor([-2.5908,  2.3052,  8.0723, -5.2359,  7.5639,  3.3462,  4.7849, -6.4288,
         3.4430, -0.4963], grad_fn=<SelectBackward0>)
tensor(4)
Would you like to see the next failed Image? (y/n): y
Test Number 259 failed: 
tensor([10.6954, -0.3559,  1.0993,  0.6267,  1.8715,  4.5637,  9.8624, -3.8007,
        -2.3825, -3.7565], grad_fn=<SelectBackward0>)
tensor(6)
Would you like to see the next failed Image? (y/n): y
Test Number 313 failed: 
tensor([-2.9812,  0.9320, -2.5912, 11.9530, -1.2624, 14.0683,  2.9889,  3.6966,
        -5.1939, -0.7126], grad_fn=<SelectBackward0>)
tensor(3)
Would you like to see the next failed Image? (y/n): y
Test Number 321 failed: 
tensor([-0.9696,  1.3357,  8.